# Notebook notes

This notebook is for testing training

# Import libs

**setup autoreload**

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**configure system path**

In [5]:
import os
import sys

# list directories where packages are stored
# note that the parent directory of te repo is added automatically
GYM_FOLDER = "gym-t99"

# get this notebook's current working directory
nb_cwd = os.getcwd()
# get name of its parent directory
nb_parent = os.path.dirname(nb_cwd)
# add packages to path
sys.path.insert(len(sys.path), nb_parent)
sys.path.insert(len(sys.path), os.path.join(nb_parent, GYM_FOLDER))

**import custom packages**

Use this code if you have not yet loaded gym_t99

In [6]:
import gym
registered = gym.envs.registration.registry.env_specs.copy()

import gym_t99
import t_net

**import custom packages**

Use this code if you have already loaded gym_t99, but now you introduced changes and want to re-load it again

In [7]:
import imp

# this code removes environment from gym's registry
env_dict = gym.envs.registration.registry.env_specs.copy()
for env in env_dict:
    if env not in registered:
        print("Remove {} from registry".format(env))
        del gym.envs.registration.registry.env_specs[env]

imp.reload(gym_t99)

Remove basic-v0 from registry
Remove t99-v0 from registry


<module 'gym_t99' from '/Users/ianconceicao/239AS_T99_RL_Agent/gym-t99/gym_t99/__init__.py'>

**import out-of-the-box packages**

In [8]:
# import matplotlib
import matplotlib as plt
# configure matplotlib
%matplotlib inline

In [9]:
import numpy as np
from collections import OrderedDict

In [10]:
# import torch
import torch
from torch import nn

# configure torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Workspace

In [11]:
# a simple random agent to play aginst and test
class RandomEnemy:
    def __init__(self):
        self.action_space = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    # this interface needs to be supported for any agent
    def action(self, state):
        return np.random.choice(self.action_space)

In [44]:
custom_gym = gym.make('gym_t99:t99-v0', num_players = 1, enemy=RandomEnemy())


In [ ]:
#Randomly fill the boards, used to test graphics
from random import randrange
custom_gym = gym.make('gym_t99:t99-v0', num_players = 100, enemy=RandomEnemy())
for player in custom_gym.state.players:
    board = player.board
    if 0 == randrange(20):
        player.place = randrange(100)
    for i in range(board.shape[0]):
        for j in range(board.shape[1]):
            if board[i][j] == 10:
                continue
            odds = 10
            if i <= 20:
                odds = 20
            
            if randrange(odds) == 0:
                board[i][j] = randrange(8)

In [45]:
frame = custom_gym.render(mode="human")
#print(frame[0])

In [16]:
custom_gym.step(7)

([(array([[10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [10, 10, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 10, 10],
          [1

In [384]:
custom_gym.state.players[0].board[11, 3] = 1

In [385]:
# custom_gym.state.players[0].board[21, 5:12] = 1
print(custom_gym.state.players[0].board)
print(custom_gym.state.players[0].piece_current.matrix)
print(custom_gym.state.players[0].piece_current.x)
print(custom_gym.state.players[0].piece_current.y)

[[10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  1  1  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  1  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  1  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  1  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  1  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  6  0  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  6  6  0  0  0  0  0  0  0 10 10 10]
 [10 10 10  0  0  6  0  0  0  0

In [334]:
custom_gym._collision(custom_gym.state.players[0].board,
                      custom_gym.state.players[0].piece_current)

False

In [333]:
custom_gym.state.players[0].piece_current.y -= 1

In [159]:
custom_gym.step(1)

(None, None, None, {})